my approach was to add both train and test together preproceess once and create create submission with both cv and normal modelling create hyperparameter tunning with optuna and integrate for better scoring

I tried neural network it's not performing well enough to outdate my score , I also create many features which are not significant , but what I thought was the key is the hyperparameter tunning ,I run it till my system shutdown , so I later use colab , and I integrate the parameter on my local server ,I think that's what score both 0.96 and 1.0 xgboost was my initial model to brought me to .91, but i duplicate my notebooks many times to change models and submit to increase my scores

# importing all necessary libraries,instantiation ,and some minimal processing

In [76]:
import datetime
import plotly.graph_objects as go
import plotly.express as px
import numpy as np
import pandas as pd
import seaborn as sns
sns.set(rc={'figure.figsize':(12,3)})
import matplotlib.pyplot as plt
plt.style.use("seaborn-whitegrid")
%matplotlib inline
from sklearn.decomposition import PCA, FastICA, TruncatedSVD
from sklearn.metrics import roc_auc_score,accuracy_score,classification_report,r2_score
from sklearn.model_selection import train_test_split,StratifiedKFold,StratifiedShuffleSplit,cross_validate,cross_val_score
from lightgbm import LGBMClassifier
from catboost import CatBoostClassifier
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler,LabelEncoder
le=LabelEncoder()
plt.style.use("seaborn-whitegrid")
import warnings
warnings.filterwarnings("ignore")
from tqdm.notebook import tqdm_notebook
import os
from sklearn.preprocessing import StandardScaler
import itertools
train = pd.read_csv('Train Dataset .csv')
test = pd.read_csv('Test Dataset.csv')

test=test.rename(columns={"id": "Id", "age": "Age","sex": "Sex"})
train['ind'] = 1
test['ind'] = 0
df= pd.concat([train,test])
obj=df.select_dtypes('object')
num=df.select_dtypes(exclude='object')


In [77]:
df

,Id,Age,Sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target,ind
0,16167,33,0,1,158,205,1,0,154,0,1.5,1,4,1,1.0,1
1,11275,53,1,2,198,154,0,1,104,0,0.8,2,1,0,0.0,1
2,13251,37,1,2,101,202,1,0,155,0,2.1,1,3,1,1.0,1
3,19921,75,0,0,113,306,1,2,88,1,4.9,0,2,2,1.0,1
4,11293,35,1,2,139,419,1,1,166,1,0.9,2,4,0,1.0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2692,14964,34,0,3,136,291,0,1,163,0,2.6,2,2,3,NaN,0
2693,16774,72,0,1,104,166,1,2,95,1,0.3,0,2,2,NaN,0
2694,18884,31,1,0,153,457,1,1,170,1,3.8,0,3,1,NaN,0
2695,10000,71,0,0,111,242,1,0,147,1,5.3,0,1,2,NaN,0


In [78]:
df.isna().sum()

Id             0
Age            0
Sex            0
cp             0
trestbps       0
chol           0
fbs            0
restecg        0
thalach        0
exang          0
oldpeak        0
slope          0
ca             0
thal           0
target      2697
ind            0
dtype: int64

In [79]:
for col in num:
    print(col, df[col].unique(), sep=':')
    print('_-_'*25)

Id:[16167 11275 13251 ... 18884 10000 17660]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
Age:[33 53 37 75 35 69 44 32 36 60 68 72 43 66 30 50 59 71 31 39 73 76 65 40
 46 58 61 47 77 45 57 74 62 29 54 34 52 67 42 41 38 51 49 64 55 63 48 70
 56]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
Sex:[0 1]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
cp:[1 2 0 3]
_-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-__-_
trestbps:[158 198 101 113 139 164 134 199 126 107 189 135 137 153 154 148 169 132
 171 175 180 138 183 130 160 174 172 187 144 140 186 188 182 177 156 157
 102 191 178  94 181 125 196 117 100 119 194 197 104 147 133  95 176 127
  99  96 108 146 161 150 152 185 173 200 195 184 121 167 129 143 165 136
 193 179 122 109 168  98 115 131  97 123 112 128 170 155 105 192 162 118
 163 141 151 149 142 111 124 114 116 166 159 103 110 120 190 106 145]
_-__-__-__-__-__-__-__-__-__-

# dropping of irrellevant columns

In [80]:
nunique_train=df.nunique().reset_index()
remove_col=nunique_train[(nunique_train[0]==0) | (nunique_train[0]==1)| (nunique_train[0]==len(train))  ]['index'].tolist()
remove_col[:5]

[]

# label encoding if necessary

for feature in col:
    df[feature] = le.fit_transform(df[feature])

# preparation for modelling

In [81]:
col=df.drop(['Id','target','ind'],axis=1).columns

In [82]:
train = df[df.ind == 1]
test = df[df.ind == 0]
from sklearn.model_selection  import train_test_split
X=train[col]
y = train['target']

In [83]:
train.target.unique()

array([1., 0.])

# optuna parameters

In [84]:
l={'lambda_l1': 0.0015159797355515795, 'lambda_l2': 0.00020625770200774653, 'num_leaves': 180, 'feature_fraction': 0.9732953081079776, 'bagging_fraction': 0.9357136597321773, 'bagging_freq': 5, 'min_child_samples': 17}
x= {'booster': 'gbtree', 'lambda': 1.3091429264953099e-06, 'alpha': 0.0013336138803455024, 'subsample': 0.6628529947638686, 'colsample_bytree': 0.9064461467673962, 'max_depth': 6, 'learning_rate': 0.11171505124760225, 'n_estimators': 327}


# modelling

In [85]:
ctb = CatBoostClassifier().fit(X,y)
xgr = XGBClassifier().fit(X,y)
lgr = LGBMClassifier(**l).fit(X,y)
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=123)


Learning rate set to 0.02408
0:	learn: 0.6488018	total: 7.16ms	remaining: 7.16s
1:	learn: 0.6092083	total: 13.8ms	remaining: 6.88s
2:	learn: 0.5738544	total: 20.4ms	remaining: 6.77s
3:	learn: 0.5431344	total: 27.5ms	remaining: 6.84s
4:	learn: 0.5080956	total: 32.8ms	remaining: 6.53s
5:	learn: 0.4811827	total: 39.3ms	remaining: 6.52s
6:	learn: 0.4591280	total: 46.1ms	remaining: 6.53s
7:	learn: 0.4371720	total: 52.5ms	remaining: 6.51s
8:	learn: 0.4152034	total: 57.8ms	remaining: 6.37s
9:	learn: 0.3962876	total: 63.2ms	remaining: 6.26s
10:	learn: 0.3832518	total: 69.7ms	remaining: 6.27s
11:	learn: 0.3701099	total: 76.1ms	remaining: 6.27s
12:	learn: 0.3600727	total: 82.7ms	remaining: 6.28s
13:	learn: 0.3489060	total: 89.2ms	remaining: 6.28s
14:	learn: 0.3400022	total: 95.6ms	remaining: 6.28s
15:	learn: 0.3326403	total: 102ms	remaining: 6.27s
16:	learn: 0.3258541	total: 108ms	remaining: 6.26s
17:	learn: 0.3198121	total: 115ms	remaining: 6.26s
18:	learn: 0.3139988	total: 121ms	remaining: 6.2

174:	learn: 0.2425050	total: 1.22s	remaining: 5.75s
175:	learn: 0.2422891	total: 1.23s	remaining: 5.75s
176:	learn: 0.2421634	total: 1.24s	remaining: 5.75s
177:	learn: 0.2420110	total: 1.24s	remaining: 5.74s
178:	learn: 0.2418524	total: 1.25s	remaining: 5.74s
179:	learn: 0.2417105	total: 1.26s	remaining: 5.73s
180:	learn: 0.2415985	total: 1.26s	remaining: 5.72s
181:	learn: 0.2414714	total: 1.27s	remaining: 5.71s
182:	learn: 0.2413572	total: 1.28s	remaining: 5.7s
183:	learn: 0.2411973	total: 1.28s	remaining: 5.69s
184:	learn: 0.2410241	total: 1.29s	remaining: 5.68s
185:	learn: 0.2408840	total: 1.3s	remaining: 5.67s
186:	learn: 0.2407550	total: 1.3s	remaining: 5.66s
187:	learn: 0.2405977	total: 1.31s	remaining: 5.65s
188:	learn: 0.2404083	total: 1.31s	remaining: 5.65s
189:	learn: 0.2402742	total: 1.32s	remaining: 5.64s
190:	learn: 0.2400471	total: 1.33s	remaining: 5.63s
191:	learn: 0.2399352	total: 1.33s	remaining: 5.62s
192:	learn: 0.2397560	total: 1.34s	remaining: 5.61s
193:	learn: 0.2

356:	learn: 0.2182044	total: 2.45s	remaining: 4.41s
357:	learn: 0.2181160	total: 2.46s	remaining: 4.41s
358:	learn: 0.2180256	total: 2.46s	remaining: 4.4s
359:	learn: 0.2179403	total: 2.47s	remaining: 4.39s
360:	learn: 0.2177643	total: 2.48s	remaining: 4.38s
361:	learn: 0.2176945	total: 2.48s	remaining: 4.38s
362:	learn: 0.2175814	total: 2.49s	remaining: 4.37s
363:	learn: 0.2174829	total: 2.5s	remaining: 4.37s
364:	learn: 0.2174065	total: 2.51s	remaining: 4.36s
365:	learn: 0.2172864	total: 2.51s	remaining: 4.36s
366:	learn: 0.2172022	total: 2.52s	remaining: 4.35s
367:	learn: 0.2171019	total: 2.53s	remaining: 4.35s
368:	learn: 0.2169146	total: 2.54s	remaining: 4.35s
369:	learn: 0.2167400	total: 2.55s	remaining: 4.34s
370:	learn: 0.2166734	total: 2.56s	remaining: 4.33s
371:	learn: 0.2165206	total: 2.56s	remaining: 4.33s
372:	learn: 0.2163910	total: 2.57s	remaining: 4.32s
373:	learn: 0.2162696	total: 2.58s	remaining: 4.31s
374:	learn: 0.2161540	total: 2.58s	remaining: 4.3s
375:	learn: 0.2

533:	learn: 0.2014636	total: 3.67s	remaining: 3.21s
534:	learn: 0.2013792	total: 3.69s	remaining: 3.21s
535:	learn: 0.2013293	total: 3.7s	remaining: 3.2s
536:	learn: 0.2012780	total: 3.7s	remaining: 3.19s
537:	learn: 0.2011972	total: 3.71s	remaining: 3.19s
538:	learn: 0.2011243	total: 3.72s	remaining: 3.18s
539:	learn: 0.2010589	total: 3.73s	remaining: 3.17s
540:	learn: 0.2010064	total: 3.73s	remaining: 3.17s
541:	learn: 0.2009399	total: 3.74s	remaining: 3.16s
542:	learn: 0.2008442	total: 3.74s	remaining: 3.15s
543:	learn: 0.2007070	total: 3.75s	remaining: 3.14s
544:	learn: 0.2006487	total: 3.76s	remaining: 3.14s
545:	learn: 0.2005735	total: 3.76s	remaining: 3.13s
546:	learn: 0.2004949	total: 3.77s	remaining: 3.12s
547:	learn: 0.2004004	total: 3.78s	remaining: 3.12s
548:	learn: 0.2003434	total: 3.78s	remaining: 3.11s
549:	learn: 0.2002635	total: 3.79s	remaining: 3.1s
550:	learn: 0.2001482	total: 3.8s	remaining: 3.09s
551:	learn: 0.2000629	total: 3.8s	remaining: 3.09s
552:	learn: 0.1999

708:	learn: 0.1882607	total: 4.91s	remaining: 2.02s
709:	learn: 0.1881649	total: 4.92s	remaining: 2.01s
710:	learn: 0.1881157	total: 4.93s	remaining: 2s
711:	learn: 0.1880704	total: 4.94s	remaining: 2s
712:	learn: 0.1879487	total: 4.95s	remaining: 1.99s
713:	learn: 0.1878647	total: 4.96s	remaining: 1.99s
714:	learn: 0.1877543	total: 4.97s	remaining: 1.98s
715:	learn: 0.1876851	total: 4.97s	remaining: 1.97s
716:	learn: 0.1875821	total: 4.98s	remaining: 1.97s
717:	learn: 0.1875238	total: 4.99s	remaining: 1.96s
718:	learn: 0.1874583	total: 4.99s	remaining: 1.95s
719:	learn: 0.1873571	total: 5s	remaining: 1.94s
720:	learn: 0.1872402	total: 5.01s	remaining: 1.94s
721:	learn: 0.1871927	total: 5.01s	remaining: 1.93s
722:	learn: 0.1871352	total: 5.02s	remaining: 1.92s
723:	learn: 0.1870819	total: 5.03s	remaining: 1.92s
724:	learn: 0.1870137	total: 5.03s	remaining: 1.91s
725:	learn: 0.1869275	total: 5.04s	remaining: 1.9s
726:	learn: 0.1868716	total: 5.05s	remaining: 1.9s
727:	learn: 0.1867671	t

877:	learn: 0.1758081	total: 6.11s	remaining: 849ms
878:	learn: 0.1757581	total: 6.12s	remaining: 842ms
879:	learn: 0.1756834	total: 6.12s	remaining: 835ms
880:	learn: 0.1756131	total: 6.13s	remaining: 828ms
881:	learn: 0.1755171	total: 6.14s	remaining: 821ms
882:	learn: 0.1754606	total: 6.14s	remaining: 814ms
883:	learn: 0.1754015	total: 6.15s	remaining: 807ms
884:	learn: 0.1753404	total: 6.16s	remaining: 800ms
885:	learn: 0.1752805	total: 6.16s	remaining: 793ms
886:	learn: 0.1752344	total: 6.17s	remaining: 786ms
887:	learn: 0.1751915	total: 6.18s	remaining: 779ms
888:	learn: 0.1751499	total: 6.18s	remaining: 772ms
889:	learn: 0.1750980	total: 6.19s	remaining: 765ms
890:	learn: 0.1750387	total: 6.2s	remaining: 758ms
891:	learn: 0.1749659	total: 6.2s	remaining: 751ms
892:	learn: 0.1749014	total: 6.21s	remaining: 744ms
893:	learn: 0.1747751	total: 6.21s	remaining: 737ms
894:	learn: 0.1746844	total: 6.22s	remaining: 730ms
895:	learn: 0.1746186	total: 6.23s	remaining: 723ms
896:	learn: 0.

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

In [86]:
y_predicted = lgr.predict(X_test)

score = (classification_report(y_test, (y_predicted)))

print("score:\n")
print(score)#739
print(accuracy_score(y_test, (y_predicted)))

[LightGBM] [Warning] feature_fraction is set=0.9732953081079776, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9732953081079776
[LightGBM] [Warning] lambda_l1 is set=0.0015159797355515795, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0015159797355515795
[LightGBM] [Warning] lambda_l2 is set=0.00020625770200774653, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00020625770200774653
[LightGBM] [Warning] bagging_fraction is set=0.9357136597321773, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9357136597321773
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5
score:

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00       443
         1.0       1.00      1.00      1.00      1967

    accuracy                           1.00      2410
   macro avg       1.00      1.00      1.00      2410
weighted avg       1.00      1

In [87]:
target = xgr.predict(test[col])
subx = pd.DataFrame({"Id": test["Id"] ,
                           "target": target})
subx.to_csv('xgr.csv', index = False)

target = np.round(lgr.predict(test[col])).astype(int)
subl = pd.DataFrame({"Id": test["Id"] ,
                           "target": target})
subl.to_csv('lgr_opt.csv', index = False)#######################################winning csvfrom 0.91-0.96-1.0

target =np.round(ctb.predict(test[col])).astype(int)
subc = pd.DataFrame({"Id": test["Id"] ,
                           "target": target})
subc.to_csv('ctb.csv', index = False)

[LightGBM] [Warning] feature_fraction is set=0.9732953081079776, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9732953081079776
[LightGBM] [Warning] lambda_l1 is set=0.0015159797355515795, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0015159797355515795
[LightGBM] [Warning] lambda_l2 is set=0.00020625770200774653, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00020625770200774653
[LightGBM] [Warning] bagging_fraction is set=0.9357136597321773, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9357136597321773
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


In [88]:
folds = StratifiedKFold(n_splits=5)
oofs  = np.zeros((len(X)))
test_predictions = np.zeros((len(test)))

for fold_, (trn_idx, val_idx) in enumerate(folds.split(X, y)):
    
    X_trn, y_trn = X.iloc[trn_idx], y.iloc[trn_idx]
    X_val, y_val = X.iloc[val_idx], y.iloc[val_idx]
    
    model =  LGBMClassifier(verbose=0)
    model.fit(X_trn, y_trn, eval_set = [(X_val, y_val)])
    
    Y_PRED =model.predict(X_val)
    oofs[val_idx] = Y_PRED
    val_score = r2_score( y_val, Y_PRED)
    val = accuracy_score( y_val, Y_PRED)
    print(25*'-_-')
    print(f'Fold {fold_+1} Val score and accuracy: {val_score},{val}')
    print(25*'-_-')
    tp = model.predict(test[col])
    tp=(np.round(tp).astype(int))

-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
Fold 1 Val score and accuracy: -0.25586874783555147,0.809719370294319
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
Fold 2 Val score and accuracy: -0.33340733340733353,0.7973990417522245
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
Fold 3 Val score and accuracy: -0.328902578902579,0.7980835044490076
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
Fold 4 Val score and accuracy: -0.20184690037631237,0.8178082191780822
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
-_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_--_-
Fold 5 Val score and accuracy: -0.2

In [89]:
target = lgr.predict(test[col])
sub = pd.DataFrame({"Id": test["Id"] ,
                           "target": tp})
sub.to_csv('lcv-1.csv', index = False)

[LightGBM] [Warning] feature_fraction is set=0.9732953081079776, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9732953081079776
[LightGBM] [Warning] lambda_l1 is set=0.0015159797355515795, reg_alpha=0.0 will be ignored. Current value: lambda_l1=0.0015159797355515795
[LightGBM] [Warning] lambda_l2 is set=0.00020625770200774653, reg_lambda=0.0 will be ignored. Current value: lambda_l2=0.00020625770200774653
[LightGBM] [Warning] bagging_fraction is set=0.9357136597321773, subsample=1.0 will be ignored. Current value: bagging_fraction=0.9357136597321773
[LightGBM] [Warning] bagging_freq is set=5, subsample_freq=0 will be ignored. Current value: bagging_freq=5


In [90]:
import xgboost as xgb
import lightgbm as lgb
import catboost as cb
import sklearn
import tensorflow as tf

# Print versions
print(f"XGBoost version: {xgb.__version__}")
print(f"LightGBM version: {lgb.__version__}")
print(f"CatBoost version: {cb.__version__}")
print(f"TensorFlow (Neural Network) version: {tf.__version__}")
print(f"Scikit-learn version: {sklearn.__version__}")


XGBoost version: 2.1.1
LightGBM version: 4.5.0
CatBoost version: 1.2.5
TensorFlow (Neural Network) version: 2.17.0
Scikit-learn version: 1.2.1
